In [90]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import requests  
from PIL import Image
import matplotlib.pyplot as plt

In [91]:
# source 이미지 가져오기
image1 = ins_get_image('source')
# plt.imshow(image1)

In [92]:
# destination 이미지 가져오기
image2 = ins_get_image('destination')
# plt.imshow(image2)

In [93]:
# 이미지를 array로 변환 후, grayscale (1차원) 변환
img = np.array(image1)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# plt.imshow(img_gray)
mask = np.zeros_like(img_gray)
img2 = np.array(image2)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# plt.imshow(img2_gray)

In [94]:
# 얼굴 검출기 및 특징점 검출기 정의
app = FaceAnalysis(allowed_modules=['detection', 'landmark_2d_106'])
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jooyeon/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/jooyeon/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/jooyeon/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jooyeon/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jooyeon/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [95]:
height, width, channels = img2.shape
img2_new_face = np.zeros((height, width, channels), np.uint8)

In [96]:
# index array에서 0번째 값 반환
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

In [99]:
insight_img = ins_get_image('source')
faces = app.get(img)
tim = insight_img.copy()

for face in faces:
    landmarks = face.landmark_2d_106
#     landmarks = np.round(landmarks).astype(np.int32)
    landmarks_points=[]
    for n in range(landmarks.shape[0]):
        p = tuple(landmarks[n])
        x = int(p[0])
        y = int(p[1])
        landmarks_points.append((x,y))
#         color = np.random.randint(0, 255, size=(3, ))
#         color = ( int (color [ 0 ]), int (color [ 1 ]), int (color [ 2 ])) 
#         cv2.circle(tim, points, 1, color, 1, cv2.LINE_AA)

    points = np.array(landmarks_points, np.int32)
#     print(points)
    # 얼굴 특징점 경계선 추출
    convexhull = cv2.convexHull(points)
    cv2.fillConvexPoly(mask, convexhull, 255)
    
    # 얼굴 특징점 경계선을 따라 얼굴 이미지 추출
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
    
    # source 얼굴 들로네 삼각망 구축
    rect = cv2.boundingRect(convexhull)
#     print(rect)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)

    # 들로네 삼각망의 triangles 인덱스 구하기 
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])


        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)

        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)

        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)

        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)